## Interface Supervisor agent

In previous notebooks, we built three specialized agents:

1. **Research agent** (Notebook 01): An expert at identifying entities in facts gathered from a news event, and then researching the background of those entities to produce data that can assist in writing a news story
2. **Article Generation agent** (Notebook 02): An expert content creator that writes high-quality news, sports, and entertainment articles following detailed journalistic guidelines. This agent focuses on clarity, accuracy, engagement, structure, balance, and technical excellence.
3. **Article Reviewer agent** (Notebook 03): A professional reviewer that analyzes articles across multiple dimensions including readability, factual accuracy, engagement, fairness, context, and ethical considerations. This agent provides detailed, constructive feedback with specific improvement suggestions.

In this notebook, we'll implement an Interface Supervisor agent that applies a multi-agent collaboration pattern to communicate between the user, Research agent, Article Generation agent, and article Reviewer agent.

## Multi-Agent Collaboration Pattern

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is an Amazon Bedrock Agents capability that enables hierarchical collaboration between agents. This pattern allows:

- A supervisor agent to coordinate specialized sub-agents
- Delegation of specific tasks to agents with specialized expertise
- Iterative improvement through feedback loops
- Complex workflows that leverage multiple AI capabilities

The supervisor agent we'll create will:
1. Receive the initial news facts regarding a news event from the user
2. Delegate entity extraction, and research collection on entities identified in the news facts to the Research Agent
2. Send the output of the Research agent to the Article Generation agent, who uses the reflection pattern to iteratively produce a new story and incorporate review feedback from the Reviewer agent.

The following architecture diagram illustrates this multi-agent collaboration workflow:

<img src="../imgs/lab7-architecture-diagram.png" width="800">

Let's get started!

First we restore the variables from the previous notebook

In [ ]:
%store -r

## Create an Interface Supervisor agent using Amazon Bedrock Agents
In this section we declare global variables that will be act as helpers during entire notebook.

In [ ]:
import boto3
import time
import uuid

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.amazon.nova-premier-v1:0'
]


## Importing helper functions
In the following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, Knowledge Bases for Bedrock creation, and accessing data on S3.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)

agents = AgentsForAmazonBedrock()

## Article Generation Supervisor Agent
Create a supervisor agent which will direct the users input to the Research agent and then send all the information to the Article Generation agent.


In [ ]:
interface_supervisor_agent_name = f"interface_supervisor_agent-{str(uuid.uuid4())[:5]}"

agent_description = "interfaces between the user, a research agent, an article generation agent and a article review agent"

instruction = """You are a supervisor agent responsible for orchestrating a news article generation workflow.
Your role is to only coordinate between agents.

Your task is to carry out an article writing workflow that involves the following:

You will be provided news facts from a news event about the article to write. Your task is to submit the unmodified facts to the researchAgent. 
The research agent will provide you with additional research about the entities it identified in the news facts.

Once the research agent is finished, submit the research information to the articleWritingAgent, which will create an article from the research and news facts. 

The article must be reviewed before returning to the user. Use the articleReviewerAgent to perform the review.

You should perform the article writing and review iteratively until the reviewer agent is satisfied with the result. 

Finally, you must return only the final article to the user. Do not provide any preemtive or additional explanation, just return the final article to the user.

# Guidelines:
- Do not modify, summarize, or filter the researchAgent agent's output before passing it to the articleWritingAgent agent.

- When working with the articleReviewAgent, always provide the the article generated by the articleWritingAgent as context without any modifications or summarization.

- Do not edit, rewrite, or enhance the articleWritingAgent agent's output before returning it to the user.
You should iterate between the writing (articleWritingAgent) and review (articleReviewAgent) process to come up with best article.

- You must not iterate the writing and review iteration processes more than 2 times. If you reached the maximum iteration, return the latest draft as the final article.

- If any agent returns an error or incomplete output, notify the user with the exact error message.
"""

supervisor_agent = agents.create_agent(
    interface_supervisor_agent_name,
    agent_description,
    instruction,
    agent_foundation_model,
    agent_collaboration='SUPERVISOR'
)

interface_supervisor_agent_id = supervisor_agent[0]
interface_supervisor_agent_alias_id = supervisor_agent[1]
interface_supervisor_agent_alias_arn = supervisor_agent[2]
agent_resourceRoleArn = supervisor_agent[3]

### Associate collaborators

In this section, we're going to recover Alias Ids of the following agents to add them as collaborators for the Interface Supervisor Agent:
1. Research Agent (Notebook 01)
2. Article Generation Supervisor (Notebook 05)

Read more about multi-agent collaboration in our [docs](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agent-collaboration.html)

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': research_agent_alias_arn,
        'sub_agent_instruction': """The research agent can extract entities based on information from a news event, verify those entities exist, and then gather information about those entities.""",
        'sub_agent_association_name': 'researchAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': article_generation_agent_alias_arn,
        'sub_agent_instruction': """The article generation agent can generate news articles for news websites given information about a news event.""",
        'sub_agent_association_name': 'articleWritingAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': article_reviewer_agent_alias_arn,
        'sub_agent_instruction': """The article review agent can review news articles for news websites given information about a news event.""",
        'sub_agent_association_name': 'articleReviewAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Associate all sub-agents with the Interface Supervisor Agent:

In [ ]:
interface_supervisor_agent_alias_id, interface_supervisor_agent_alias_arn = agents.associate_sub_agents(
    interface_supervisor_agent_id, sub_agents_list
)

## Invoking multi-agent collaboration

With our supervisor agent created and connected to both specialized agents, we can now demonstrate the power of multi-agent collaboration. The supervisor agent will orchestrate the entire workflow:

1. It will first send the user input to the Research Agent for entity recognition, verification and additional research
2. The results of the research and the original news facts will then be sent to the Article Generation agent
3. The Article Generation agent will first create the article
4. Then send the draft to the Article Reviewer agent for detailed feedback
5. The Article Review agent will return the draft with feedback to the Article Generation agent for refinement
6. Repeat this cycle up to two times to continuously improve the article quality
7. Finally deliver the polished article to the user

This collaborative approach combines the strengths of all specialized agents - the researching capability of the Research agent, the creative writing capabilities of the Article Generation agent and the critical analysis skills of the Article Reviewer agent - resulting in higher quality content than either agent could produce independently.

We can now set a dictionary with agents names for better visualization of the traces

In [ ]:
multi_agent_names = {
    f"{research_agent_id}/{research_agent_alias_id}": research_agent_name,
    f"{article_generation_agent_id}/{article_generation_alias_id}": article_generation_agent_name,
    f"{article_reviewer_agent_id}/{article_reviewer_agent_alias_id}": article_reviewer_agent_name,
    f"{interface_supervisor_agent_id}/{interface_supervisor_agent_alias_id}": interface_supervisor_agent_name
}

## Testing Agent
Let's test this multi-agent system by providing the Interface Supervisor agent with mock news facts that have been collected by a journalist.

In [ ]:
news_facts = """NeuraHealth Solutions announced its new medical diagnostic platform called "MediScan" at their annual developer conference yesterday.
The system demonstrated 94% accuracy in early disease detection across a trial of 12,000 patients.
Dr. Eliza Chen, Chief Medical Officer at NeuraHealth, revealed the system was trained on 50 million anonymized patient records.
NeuraHealth CEO Marcus Williams stated the company invested $450 million in research and development over three years.
The platform will be piloted at five major hospital networks starting next month.
Senior Vice President of Product Development, Raj Patel, confirmed that FDA approval is expected by the third quarter.
Initial focus areas include cardiovascular disease, diabetes, and early cancer detection.
"""

In [ ]:
article_query_prompt = news_facts

article_query_prompt

In [ ]:
%%time
session_id:str = str(uuid.uuid4())

interface_supervisor_agent_response = agents.invoke(
    article_query_prompt,
    interface_supervisor_agent_id,
    session_id=session_id,
    enable_trace=True,
    trace_level = "all",
    multi_agent_names=multi_agent_names
)

Here's the final output:

In [ ]:
print(interface_supervisor_agent_response)

# Summary

In this lab, we demonstrated how to implement a powerful **Multi-Agent Collaboration** pattern using Amazon Bedrock Agents. This pattern enables complex workflows through the orchestration of specialized agents working together to achieve superior results.

Our implementation consisted of four key components:

1. **Research Agent**: A specialized agent which can extract entities from news facts and provide detailed research on each entity to help write a balanced news article.

2. **Article Generation Agent**: A specialized content creator that produces high-quality articles following detailed journalistic guidelines across multiple dimensions including clarity, accuracy, engagement, structure, and technical excellence.

3. **Article Reviewer Agent**: A professional reviewer that analyzes content across multiple dimensions and provides detailed, constructive feedback with specific improvement suggestions.

4. **Interface Supervisor Agent**: This orchestrates the whole workflow of passing information from the user to the research and generation agents.

The multi-agent collaboration pattern demonstrated several key advantages:

- **Specialized Expertise**: Each agent focuses on its core competency - content creation or critical review
- **Iterative Improvement**: The feedback loop enables continuous refinement of the content
- **Workflow Automation**: The supervisor agent manages the entire process without requiring user intervention
- **Quality Enhancement**: The final output benefits from multiple perspectives and expertise

